<a href="https://cognitiveclass.ai"><img src = "https://ibm.box.com/shared/static/9gegpsmnsoo25ikkbl4qzlvlyjbgxs5x.png" width = 400> </a>

<h1 align=center><font size = 5>Segmenting and Clustering Neighborhoods in Toronto</font></h1>

#### First we import all the libraries

In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup



#### Load data from wikipedia
Then we load all the data from the wikipiedia. We use request and beautiful soup

In [3]:
wikiurl="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
table_class="wikitable sortable jquery-tablesorter"
response=requests.get(wikiurl)

soup = BeautifulSoup(response.text, 'html.parser')
indiatable=soup.find('table',{'class':"wikitable"})


#### Convert data to dataframe

In [4]:
df=pd.read_html(str(indiatable))
df=pd.DataFrame(df[0])
display(df.head())

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


### Drop the cells where Borough is not assigned, and if Borough is assigned but Neighbourhood no, neighbourhood will get the value from Borough

In [5]:
index = df[ (df['Borough'] == 'Not assigned')  ].index
df.drop(index,inplace=True)
df=df.reset_index(drop=True)
for i,row in df.iterrows():
    if(row['Neighbourhood']=='Not assigned'):
        row['Neighbourhood']=row['Borough']

display(df)

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


### Call function shape to check the results

In [6]:
df.shape

(103, 3)

# Second Part

### Install Geocoder

In [7]:
! pip install geocoder

     |████████████████████████████████| 102kB 4.1MB/s 


### Import Geocoder

In [8]:
import geocoder # import geocoder


### Function to get latitudes and longitudes

In [9]:
def get_coordinates(postal_code):
  lat_lng_coords = None

  # loop until you get the coordinates
  while(lat_lng_coords is None):
    g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
    lat_lng_coords = g.latlng
  print("Postal Code ", postal_code)
  latitude = lat_lng_coords[0]
  longitude = lat_lng_coords[1]
  return latitude, longitude

### We get all the unique postal codes

In [10]:
unique_postal_codes=df['Postal Code'].unique()
print(unique_postal_codes)


['M3A' 'M4A' 'M5A' 'M6A' 'M7A' 'M9A' 'M1B' 'M3B' 'M4B' 'M5B' 'M6B' 'M9B'
 'M1C' 'M3C' 'M4C' 'M5C' 'M6C' 'M9C' 'M1E' 'M4E' 'M5E' 'M6E' 'M1G' 'M4G'
 'M5G' 'M6G' 'M1H' 'M2H' 'M3H' 'M4H' 'M5H' 'M6H' 'M1J' 'M2J' 'M3J' 'M4J'
 'M5J' 'M6J' 'M1K' 'M2K' 'M3K' 'M4K' 'M5K' 'M6K' 'M1L' 'M2L' 'M3L' 'M4L'
 'M5L' 'M6L' 'M9L' 'M1M' 'M2M' 'M3M' 'M4M' 'M5M' 'M6M' 'M9M' 'M1N' 'M2N'
 'M3N' 'M4N' 'M5N' 'M6N' 'M9N' 'M1P' 'M2P' 'M4P' 'M5P' 'M6P' 'M9P' 'M1R'
 'M2R' 'M4R' 'M5R' 'M6R' 'M7R' 'M9R' 'M1S' 'M4S' 'M5S' 'M6S' 'M1T' 'M4T'
 'M5T' 'M1V' 'M4V' 'M5V' 'M8V' 'M9V' 'M1W' 'M4W' 'M5W' 'M8W' 'M9W' 'M1X'
 'M4X' 'M5X' 'M8X' 'M4Y' 'M7Y' 'M8Y' 'M8Z']


### We create a new dataframe with all the postal codes and their latitude

#### The geocoder didn't work, so I used the .csv later on

In [17]:
Lat_lon_df= pd.DataFrame(columns={"Postal Code","Latitude","Longitude"})
Lat_lon=[]
for postal_code in unique_postal_codes:
  latitude,longitude=get_coordinates(postal_code)
  Lat_lon.append([postal_code,latitude,longitude])
 
Lat_lon_df= pd.DataFrame(Lat_lon,columns=["Postal Code","Latitude","Longitude"])
display(Lat_lon_df)

In [14]:
from google.colab import drive
drive.mount('/content/gdrive')
Directory='/content/gdrive/MyDrive/Geospatial_Coordinates.csv'
Lat_lon_df= pd.read_csv(Directory)
Lat_lon_df.head()

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### We perform a inner join in both frames with the fiel Postal code

In [19]:
Final_df=pd.merge(df,Lat_lon_df,on='Postal Code')
display(Final_df)

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509
